<a href="https://colab.research.google.com/github/jwr0218/analysis_reviews_restaurant_with_Korean_English/blob/main/ENGLISH_TM_NB_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install lda

In [ ]:

!pip install konlpy

In [ ]:
import nltk
import re
import collections
import itertools
import lda
import requests
import csv
import time
import math
import operator
import numpy as np
import pandas as pd
from collections import defaultdict
from pandas import read_table
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from konlpy  import tag 

from nltk.tokenize import sent_tokenize

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')



In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Tripadvisor_final (1).csv', encoding = 'utf-8')

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
data

In [ ]:
data

data['evaluation'] = data.apply(lambda x : 0 if x['evaluation'] <25  else x['evaluation'],axis = 1)

data['evaluation'] = data.apply(lambda x : 1 if x['evaluation'] >= 40 else x['evaluation'],axis = 1)




data = data.dropna(subset=['evaluation'])

In [ ]:
data

In [ ]:
total_review = data['review']
positive_review =  data[data['evaluation']==1]['review']


negative_review = data[data['evaluation']==0]['review']


tot_review = total_review.values.tolist()
pos_review = positive_review.values.tolist()
neg_review = negative_review.values.tolist()

In [ ]:
total_review

In [ ]:
data

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer


class SentenceTokenizer(object):
    
    def __init__(self):
        self.n = WordNetLemmatizer

        self.s = PorterStemmer()
        self.stopword = ['the','and', 'to', 'of', 'in', 'it', 'for', 'with', 'is', 'wa', 'that', 'but', 'on', 'my', 'this', 'you', 'at', 'not', 'are', 'have', 'food', 'restaurant', 'they',
 'so', 'from', 'be', 'place', 'good', 'we', 'had', 'were' 'seoul', 'if','all','which','or','there','like','just','were','the','it','very','about','some','do','we','when','있는','would','menu'
 ,'make','your','너무','other','here','than','no','we','out','by','can','an','정말','korean'
 ,'will','다른','korean','서울시','맛은','합니다','강남구','their','위치한','the','하지만','any','after'
 ,'this','there','they']
        
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    
    
    
    
    def get_find_stem(self, sentences):
        adj = []
        stopword_1 = ['the','and', 'to', 'of', 'in', 'it', 'for', 'with', 'is', 'wa', 'that', 'but', 'on', 'my', 'this', 'you', 'at', 'not', 'are', 'have', 'food', 'restaurant', 'they',
 'so', 'from', 'be', 'place', 'good', 'we', 'had', 'were' 'seoul', 'if','all','which','or','there','like','just','were','the','it','very','about','some','do','we','when','있는','would','menu'
 ,'make','your','너무','other','here','than','no','we','out','by','can','an','정말','korean'
 ,'will','다른','korean','서울시','맛은','합니다','강남구','their','위치한','the','하지만','any','after'
 ,'this','there','they'
 ]
        count = dict()
        find = []
        for sentence in sentences:
            
            if sentence is not '':
                
                
                find = [self.s.stem(a) for a in word_tokenize(sentence)
                                           if  self.s.stem( a ) not in stopword_1 ]
                find = [w.lower() for w in find if not w in stopwords.words('english')]
                adj.append(' '.join(find))
                
                
                for i in find:
                    try:
                        count[i] = count[i] + 1
                    except :
                        count[i] = 1       
        print(count)
        return adj ,count

In [ ]:
tokenizer = SentenceTokenizer()


total_tokenized , total_count = tokenizer.get_find_stem(total_review)

positive_tokenized ,positive_count = tokenizer.get_find_stem(positive_review)

negative_tokenized, negative_count = tokenizer.get_find_stem(negative_review)


In [ ]:
total_tokenized

In [ ]:
service_list = ['servic', 'attit', 'staff', 'waiter', 'serv', 'english']
taste_list = ['dish','pizza','pasta','food','meal','jungsik','bottarga','drink','taste','dessertch','grill','appetis','plate','soup','lunch','crab','stale','fish','tast'
'dinner','set','bibimbap','gimbap','lobster','seafood','bbq','branch']
cost_list = []

atmosphere_list = ["music", "atmosphere", "conversation", "deco", "girlfriend", "musician", "jazz", "piano", "vocalist", "family",'transport']


service_good_feature = {'better':0,'excellent':0,'kind':0,'authentic':0,'consistent':0,'spectacular':0,'excellent':0,'outstanding':0,'great':0,'impeccable':0,'fine':0,'nice':0,'impressive':0,'perfect':0,'good':0,'incredible':0,'best':0,'outstanding':0,'timely':0,'friendly':0,
                        'awesome':0,'attent':0 ,'profes':0, 'understan':0, 'pleasant':0, 'on point':0, 'superb':0, 'efficient':0,'thoughtful':0, 'fantastic':0, 'lovely':0, 'spot on':0,'help':0, 'prompt':0, 'enjoyed':0, 'top':0, 'wonderful':0, 'detail':0, 'flawless':0, 'quick':0, 'courteous':0,'fast':0,'fluent':0, 'ease':0, 'speak':0, 'thoughtful':0}


service_bad_feature = {'let down':0,'flaw':0,'notatten':0,'cold':0,'overat':0,'mediocre':0,'quic':0,'ordi':0,'passable':0,'no help':0,'unimpre':0,'shortcom':0,'slow':0,'terrible':0,'worst':0,'rude':0,'unfriendly':0,'unacceptable':0,'bad':0,'poor':0,'horrible':0,'lifeless':0,
                        'brusque':0,'notgood':0,'average':0, 'hard communicate':0, 'limit':0, 'notfluent':0}
# 서비스 좋음 / 안좋음 
#맛


taste_good_feature = { 'delici':0, 'amaz':0,'spectacular':0,'super':0,'love':0,'perfectli':0,'perfect':0,'better':0,'good':0,'wonderful':0,'comfort':0,'crispi':0,
                      'great':0,'fast':0,'popular':0,'cool':0}

taste_bad_feature = {'overpriced':0,'awful':0,'wrong':0,'overprice':0,'terribl':0,'overr':0,'overcook':0,'disappointed':0,'unrefined':0 ,'regrect':0,'cold':0,'unfortun':0,
                     'poor':0, 'worst' : 0 , 'empti':0, 'tiny':0,'horrible':0, 'expens':0,'miss':0,'dri':0, 'bad':0, 'notgood':0
                     }
#느낌 입감 
cost_good_feature = {'괜찮':0,'착하다':0,'저렴':0,'적당':0,'싸다':0,'좋다':0,'합리적':0,'훌륭':0,'최고':0,'만족':0,'마음':0,'든든':0,'알맞다':0,'무난':0,'괜춘':0,'최상':0,'낮':0
                     ,'많':0,'적당':0,'푸짐':0,'괜찮다':0,'넉넉':0,'충분':0    
                     }

cost_bad_feature ={'비싸':0,'있다':0,'나쁘':0,'사악':0,'비효율':0,'높다':0,'부담':0,'아쉽':0,'별로':0,'그닥':0,'그다지':0,'쎄':0,'높':0,'거품':0,'눈물':0,
                   '부족':0, '별로':0,'적다':0,'작다':0,'아쉽':0,'적다':0,'다소':0,'별로':0  
                   }
# 특징 

atmosphere_good_feature ={'wonder':0,'luv':0, 'amaz':0, 'great':0,'awesome':0, 'outstand':0,  'relaxed':0, 'calm':0, 'cozy':0, 'fancy':0, 'funny':0, 
                          'ok':0, 'pleasant':0, 'relax':0,'excellent':0,'cool':0, 'perfect':0,  'couple':0, 'roman':0, 
                          'cute':0,'comfort':0,'enjoy':0,'fine':0, 'propos':0,  'quiet':0,  'classy':0, 'snooty':0 , 'private':0, 'trendy':0, 'celebrating':0, 'appriciated':0,
                          'peaceful':0, 'tranquil':0, 'brilliant':0, 'warm':0, 'basic':0, 'happy':0, 'stylish':0}


atmosphere_bad_feature ={'sadly':0, 'disappoined':0, 'Not':0, 'stuffy':0, 'wide':0, 'rare':0,'regret':0,'wrong':0}
#분위기
#transport poor / manag aploget




In [ ]:
def keyword_count(review,dic):
  
  for i in dic.keys():


   
    
      
    if i in review:


      dic[i] +=1
  return dic 
        

In [ ]:
def condition_keyword_count(review,dic,lst):


  find = word_tokenize(review)
  

  for i in find :
    if i in lst:
      dic =keyword_count(review,dic)
  return dic       

In [ ]:
keyword_count(total_tokenized[0],service_good_feature)

In [ ]:
preprocessed_data_service = pd.DataFrame()
preprocessed_data_taste = pd.DataFrame()
preprocessed_data_cost = pd.DataFrame()
preprocessed_data_atmosphere = pd.DataFrame()

for token in total_tokenized:
  
  
  service_good_feature = {'better':0,'excellent':0,'kind':0,'authentic':0,'consistent':0,'spectacular':0,'excellent':0,'outstanding':0,'great':0,'impeccable':0,'fine':0,'nice':0,'impressive':0,'perfect':0,'good':0,'incredible':0,'best':0,'outstanding':0,'timely':0,'friendly':0,
                          'awesome':0,'attent':0 ,'profes':0, 'understan':0, 'pleasant':0, 'on point':0, 'superb':0, 'efficient':0,'thoughtful':0, 'fantastic':0, 'lovely':0, 'spot on':0,'help':0, 'prompt':0, 'enjoyed':0, 'top':0, 'wonderful':0, 'detail':0, 'flawless':0, 'quick':0, 'courteous':0,'fast':0,'fluent':0, 'ease':0, 'speak':0, 'thoughtful':0}


  service_bad_feature = {'let down':0,'flaw':0,'notatten':0,'cold':0,'overat':0,'mediocre':0,'quic':0,'ordi':0,'passable':0,'no help':0,'unimpre':0,'shortcom':0,'slow':0,'terrible':0,'worst':0,'rude':0,'unfriendly':0,'unacceptable':0,'bad':0,'poor':0,'horrible':0,'lifeless':0,
                          'brusque':0,'notgood':0,'average':0, 'hard communicate':0, 'limit':0, 'notfluent':0}
  # 서비스 좋음 / 안좋음 
  #맛


  taste_good_feature = { 'delici':0, 'amaz':0,'spectacular':0,'super':0,'love':0,'perfectli':0,'perfect':0,'better':0,'good':0,'wonderful':0,'comfort':0,'crispi':0,
                        'great':0,'fast':0,'popular':0,'cool':0}

  taste_bad_feature = {'overpriced':0,'awful':0,'wrong':0,'overprice':0,'terribl':0,'overr':0,'overcook':0,'disappointed':0,'unrefined':0 ,'regrect':0,'cold':0,'unfortun':0,
                      'poor':0, 'worst' : 0 , 'empti':0, 'tiny':0,'horrible':0, 'expens':0,'miss':0,'dri':0, 'bad':0, 'notgood':0
                      }
  #느낌 입감 
  cost_good_feature = {'nice':0, 'better':0, 'good':0, 'ok':0, 'recommended':0, 'reasonable':0, 'cheap':0, 'half':0, 'affordable':0, 'friendly':0, 'worth':0, 'important':0, 'right':0,
                      'excellent':0, 'well':0, 'little':0, 'standard':0, 'extra':0, 'enjoy':0, 'fairly':0, 'amazing':0, 'satisfied':0
                      }

  cost_bad_feature ={ 'double':0, 'overpriced':0, 'overcharge':0, 'slightly':0, 'expensive':0, 'higher':0, 'pricecy':0, 'ridiculous':0, 'unacceptable':0, 'usual':0, 'steep':0, 'exorbitant':0,
                     'outrageous':0, 'surprised':0
                    }
  # 특징 

  atmosphere_good_feature ={'wonder':0,'luv':0, 'amaz':0, 'great':0,'awesome':0, 'outstand':0,  'relaxed':0, 'calm':0, 'cozy':0, 'fancy':0, 'funny':0, 
                            'ok':0, 'pleasant':0, 'relax':0,'excellent':0,'cool':0, 'perfect':0,  'couple':0, 'roman':0, 
                            'cute':0,'comfort':0,'enjoy':0,'fine':0, 'propos':0,  'quiet':0,  'classy':0, 'snooty':0 , 'private':0, 'trendy':0, 'celebrating':0, 'appriciated':0,
                            'peaceful':0, 'tranquil':0, 'brilliant':0, 'warm':0, 'basic':0, 'happy':0, 'stylish':0}


  atmosphere_bad_feature ={'sadly':0, 'disappoined':0, 'Not':0, 'stuffy':0, 'wide':0, 'rare':0,'regret':0,'wrong':0, 'noisy':0}
#분위기
#transport poor / manag aploget
  #분위기
  service_good_feature = condition_keyword_count(token,service_good_feature,service_list)
  service_bad_feature = condition_keyword_count(token,service_bad_feature,service_list)
  service_good_feature_df = pd.DataFrame(service_good_feature,index = [0])
  service_bad_feature_df = pd.DataFrame(service_bad_feature,index = [0])
  
  preprocessed_data_service_concated = pd.concat([service_good_feature_df,service_bad_feature_df],axis = 1)


  taste_good_feature = condition_keyword_count(token,taste_good_feature,taste_list)
  taste_bad_feature = condition_keyword_count(token,taste_bad_feature,taste_list)
  taste_good_feature_df = pd.DataFrame(taste_good_feature,index = [0])
  taste_bad_feature_df = pd.DataFrame(taste_bad_feature,index = [0])
  
  
  preprocessed_data_taste_concated = pd.concat([taste_good_feature_df,taste_bad_feature_df],axis = 1)

  
  
  cost_good_feature = condition_keyword_count(token,cost_good_feature ,cost_list)
  cost_bad_feature = condition_keyword_count(token,cost_bad_feature ,cost_list)
  cost_good_feature_df = pd.DataFrame(cost_good_feature,index = [0])
  cost_bad_feature_df = pd.DataFrame(cost_bad_feature,index = [0])
  
  preprocessed_data_cost_concated = pd.concat([cost_good_feature_df,cost_bad_feature_df],axis = 1)

  
  atmosphere_good_feature = condition_keyword_count(token,atmosphere_good_feature,atmosphere_list)
  atmosphere_bad_feature = condition_keyword_count(token,atmosphere_bad_feature,atmosphere_list)
  atmosphere_good_feature_df = pd.DataFrame(atmosphere_good_feature,index = [0])
  atmosphere_bad_feature_df = pd.DataFrame(atmosphere_bad_feature,index = [0])

  preprocessed_data_atmosphere_concated = pd.concat([atmosphere_good_feature_df,atmosphere_bad_feature_df],axis = 1)



  preprocessed_data_service = pd.concat([preprocessed_data_service,preprocessed_data_service_concated],ignore_index=True )
  preprocessed_data_taste = pd.concat([preprocessed_data_taste,preprocessed_data_taste_concated],ignore_index=True )
  preprocessed_data_cost = pd.concat([preprocessed_data_cost,preprocessed_data_cost_concated],ignore_index=True )
  preprocessed_data_atmosphere = pd.concat([preprocessed_data_atmosphere,preprocessed_data_atmosphere_concated],ignore_index=True )





In [ ]:
preprocessed_data_service

In [ ]:
preprocessed_data_service['evaluation'] = data['evaluation'].copy()
preprocessed_data_service = preprocessed_data_service[preprocessed_data_service['evaluation']!='괜찮다']


preprocessed_data_taste['evaluation'] = data['evaluation'].copy()

preprocessed_data_taste = preprocessed_data_taste[preprocessed_data_taste['evaluation']!='괜찮다']

preprocessed_data_cost['evaluation'] = data['evaluation'].copy()

preprocessed_data_cost = preprocessed_data_cost[preprocessed_data_cost['evaluation']!='괜찮다']
preprocessed_data_atmosphere['evaluation'] = data['evaluation'].copy()

preprocessed_data_atmosphere = preprocessed_data_atmosphere[preprocessed_data_atmosphere['evaluation']!='괜찮다']

In [ ]:
preprocessed_data_atmosphere.iloc[:,:-1]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
def elbow(X):
  sse = []
  silhouette = []
  for i in range(2,20):
    km  = KMeans(n_clusters = i , init = 'k-means++' , random_state =0 )
    km.fit(X)
    sse.append(km.inertia_)
    sil = silhouette_score(X,km.labels_   )
    silhouette.append(sil)
  plt.figure(1)
  plt.plot(range(2,20),sse , marker = 'o')
  plt.xlabel('number of cluster')
  plt.ylabel('SSE')
  plt.figure(2)
  plt.plot(range(2,20),silhouette , marker = 'x')
  plt.xlabel('number of cluster')
  plt.ylabel('silhouette')

  plt.show
  

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import NearMiss
import pandas as pd

import math 

# 정확도 계산을 위한 함수

list = []
coeff = []

df = preprocessed_data_service
for i in range(1000):
    data_ran = df[df['evaluation'].isin([1])]

    data_ran = data_ran.sample(n=200, random_state = 10)

    data_zero = df[df['evaluation'].isin([0])]

    df  = pd.concat([data_ran, data_zero])
    
   
    
    


    x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

    mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
    mod.fit(x_train, y_train)

    predicted = mod.predict(x_test)

    coeff.append( mod.feature_log_prob_ )


 
    list.append(accuracy_score(y_test, predicted))

import numpy as np

list_np = np.array(list)
a = np.mean(list_np)
b = np.mean(coeff,axis = 0)

print('score is',a)
print(b)

In [ ]:

data_for_kmeans = df.values
pca = PCA(n_components=7)

data_for_kmeans = pca.fit_transform(data_for_kmeans)
data_for_kmeans


elbow(data_for_kmeans)


In [ ]:

local_km = KMeans(n_clusters = 6 )
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label.copy()


In [ ]:
pca = PCA(n_components=2)

data_for_kmeans = pca.fit_transform(data_for_kmeans)



plt.scatter(data_for_kmeans[:,0],data_for_kmeans[:,1],c = df['label_clustered'])
plt.show()

In [ ]:



coeff_service = pd.DataFrame(np.exp(b), columns= preprocessed_data_service.columns[:-1] )




In [ ]:
coeff_service

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

a = coeff_service.iloc[0].to_dict()
b = coeff_service.iloc[1].to_dict()

word_cloud_dict1 = collections.Counter(a)
wc1 = WordCloud(max_font_size = 200, background_color="White", width=800, height=800).generate_from_frequencies(word_cloud_dict1)

word_cloud_dict2 = collections.Counter(b)
wc2 = WordCloud(max_font_size = 200, background_color="White", width=800, height=800).generate_from_frequencies(word_cloud_dict2)

plt.figure(figsize=(20,20))
#plt.imshow(wc1)
plt.imshow(wc2)
plt.axis('off')
plt.title('Service good')
plt.tight_layout(pad=0)
plt.show()




In [ ]:

list = []
coeff = []

df = preprocessed_data_taste
for i in range(1000):
    data_ran = df[df['evaluation'].isin([1])]

    data_ran = data_ran.sample(n=200, random_state = 10)

    data_zero = df[df['evaluation'].isin([0])]

    df  = pd.concat([data_ran, data_zero])
    
   
    
    


    x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

    mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
    mod.fit(x_train, y_train)

    predicted = mod.predict(x_test)

    coeff.append( mod.feature_log_prob_ )


 
    list.append(accuracy_score(y_test, predicted))

import numpy as np

list_np = np.array(list)
a = np.mean(list_np)
b = np.mean(coeff,axis = 0)

print('score is',a)
print(b)

In [ ]:

data_for_kmeans = df.values
pca = PCA(n_components=7)

data_for_kmeans = pca.fit_transform(data_for_kmeans)
data_for_kmeans


elbow(data_for_kmeans)


In [ ]:

local_km = KMeans(n_clusters = 6 )
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label.copy()


In [ ]:
pca = PCA(n_components=2)

data_for_kmeans = pca.fit_transform(data_for_kmeans)



plt.scatter(data_for_kmeans[:,0],data_for_kmeans[:,1],c = df['label_clustered'])
plt.show()

In [ ]:

coeff_taste = pd.DataFrame(np.exp(b), columns= preprocessed_data_taste.columns[:-1] )




In [ ]:
coeff_taste

In [ ]:
a = coeff_taste.iloc[0].to_dict()
b = coeff_taste.iloc[1].to_dict()

word_cloud_dict1 = collections.Counter(a)
wc1 = WordCloud(max_font_size = 200, background_color="White", width=800, height=800).generate_from_frequencies(word_cloud_dict1)

word_cloud_dict2 = collections.Counter(b)
wc2 = WordCloud(max_font_size = 200, background_color="White", width=800, height=800).generate_from_frequencies(word_cloud_dict2)

plt.figure(figsize=(20,20))
#plt.imshow(wc1)
plt.imshow(wc2)
plt.axis('off')
plt.title('Taste good')
plt.tight_layout(pad=0)
plt.show()

In [ ]:

list = []
coeff = []

df = preprocessed_data_cost
for i in range(1000):
    data_ran = df[df['evaluation'].isin([1])]

    data_ran = data_ran.sample(n=200, random_state = 10)

    data_zero = df[df['evaluation'].isin([0])]

    df  = pd.concat([data_ran, data_zero])
    
   
    
    


    x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

    mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
    mod.fit(x_train, y_train)

    predicted = mod.predict(x_test)

    coeff.append( mod.feature_log_prob_ )


 
    list.append(accuracy_score(y_test, predicted))

import numpy as np

list_np = np.array(list)
a = np.mean(list_np)
b = np.mean(coeff,axis = 0)

print('score is',a)
print(b)

In [ ]:

data_for_kmeans = df.values
pca = PCA(n_components=7)

data_for_kmeans = pca.fit_transform(data_for_kmeans)
data_for_kmeans


elbow(data_for_kmeans)

In [ ]:

local_km = KMeans(n_clusters = 6 )
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label.copy()


In [ ]:
pca = PCA(n_components=2)

data_for_kmeans = pca.fit_transform(data_for_kmeans)



plt.scatter(data_for_kmeans[:,0],data_for_kmeans[:,1],c = df['label_clustered'])
plt.show()

In [ ]:

coeff_cost = pd.DataFrame(np.exp(b), columns= preprocessed_data_cost.columns[:-1] )




In [ ]:
coeff_cost

In [ ]:
a = coeff_cost.iloc[0].to_dict()
b = coeff_cost.iloc[1].to_dict()

word_cloud_dict1 = collections.Counter(a)
wc1 = WordCloud(max_font_size = 200, background_color="White", width=800, height=800).generate_from_frequencies(word_cloud_dict1)

word_cloud_dict2 = collections.Counter(b)
wc2 = WordCloud(max_font_size = 200, background_color="White", width=800, height=800).generate_from_frequencies(word_cloud_dict2)

plt.figure(figsize=(20,20))
#plt.imshow(wc1)
plt.imshow(wc2)
plt.axis('off')
plt.title('Cost good')
plt.tight_layout(pad=0)
plt.show()

In [ ]:

list = []
coeff = []

df = preprocessed_data_atmosphere
for i in range(1000):
    data_ran = df[df['evaluation'].isin([1])]

    data_ran = data_ran.sample(n=200, random_state = 10)

    data_zero = df[df['evaluation'].isin([0])]

    df  = pd.concat([data_ran, data_zero])
    
   
    
    


    x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

    mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
    mod.fit(x_train, y_train)

    predicted = mod.predict(x_test)

    coeff.append( mod.feature_log_prob_ )


 
    list.append(accuracy_score(y_test, predicted))

import numpy as np

list_np = np.array(list)
a = np.mean(list_np)
b = np.mean(coeff,axis = 0)

print('score is',a)
print(b)

In [ ]:

data_for_kmeans = df.values
pca = PCA(n_components=7)

data_for_kmeans = pca.fit_transform(data_for_kmeans)
data_for_kmeans


elbow(data_for_kmeans)

In [ ]:

local_km = KMeans(n_clusters = 7 )
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label.copy()


In [ ]:
pca = PCA(n_components=2)

data_for_kmeans = pca.fit_transform(data_for_kmeans)



plt.scatter(data_for_kmeans[:,0],data_for_kmeans[:,1],c = df['label_clustered'])
plt.show()

In [ ]:

local_km = KMeans(n_clusters = 6 )
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label.copy()


In [ ]:

coeff_atmosphere = pd.DataFrame(np.exp(b), columns= preprocessed_data_atmosphere.columns[:-1] )




In [ ]:
coeff_atmosphere

In [ ]:
a = coeff_atmosphere.iloc[0].to_dict()
b = coeff_atmosphere.iloc[1].to_dict()

word_cloud_dict1 = collections.Counter(a)
wc1 = WordCloud(max_font_size = 200, background_color="White", width=800, height=800).generate_from_frequencies(word_cloud_dict1)

word_cloud_dict2 = collections.Counter(b)
wc2 = WordCloud(max_font_size = 200, background_color="White", width=800, height=800).generate_from_frequencies(word_cloud_dict2)

plt.figure(figsize=(20,20))
#plt.imshow(wc1)
plt.imshow(wc2)
plt.axis('off')
plt.title('atmostphere bad')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
coeff_mean = np.mean(coeff,axis = 0)


In [ ]:
type(coeff_mean)

In [ ]:

coeff_mean = pd.DataFrame(np.exp(coeff_mean), columns= preprocessed_data_atmosphere.columns[:-1] )

In [ ]:
coeff_mean

In [ ]:
df = preprocessed_data_service


data_ran = df[df['evaluation'].isin([1])]

data_ran = data_ran.sample(n=390, random_state = 10)

data_zero = df[df['evaluation'].isin([0])]

df  = pd.concat([data_ran, data_zero])
    
   
    
    


x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
mod.fit(x_train, y_train)

predicted = mod.predict(x_test)
mod.score(y_test,predicted)



In [ ]:
mod.feature_log_prob_





In [ ]:
ccc = '아 테스트 123123 keyword 상황 파악 어렵  '
re.findall('keyword +[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s', ccc)


In [ ]:

def turn_feature_df(dic):
  dic2 = {}
  for i in dic.keys():
    for j in dic[i].keys():
      dic2[i+"_"+j] = dic[i][j]

  return dic2

In [ ]:
from google.colab import files
preprocessed_data.to_csv('data_for_training.csv')
#taste_good_emotion
files.download('data_for_training.csv')

In [ ]:
preprocessed_data

In [ ]:
taste_good_emotion

In [ ]:
def ngram2_count(review,dic):



  for i in dic.keys():

    a = re.findall(i +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s',review)
   
    b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+'+i,review)
      
    if len(a)>0:
      print(a,"a 입니다.")
      splited_a = a[0].split()
      print(splited_a)
      try:
        dic[i][splited_a[1]] += 1

      except :

        pass
      try:
        dic[i][splited_a[2]] += 1

      except :
        pass
      try:
        dic[i][splited_a[3]] += 1

      except :
        pass
      
      try:
        dic[i][splited_a[4]] += 1

      except :
        pass
          
    elif len(b)>0:
        
        
      print(b,'B 입니다.')
      splited_b = b[0].split()
      print(splited_b)
      try:
        dic[i][splited_b[0]] += 1

      except :
        pass
      try:
        dic[i][splited_b[1]] += 1
      except :
        pass
      try:
        dic[i][splited_b[2]] += 1

      except :
        pass
      try:
        dic[i][splited_b[3]] += 1

      except :
        pass
        
          
  return dic

      
    
    

      


